# Pr12-2: Colab上でのblenderレンダリング


In [1]:
#@title セットアップ
#@markdown 以下の項目を設定．

path_to_blend = 'Pr12_1_LightPath.blend'  #@param {type: "string"}
output_path = 'outputs'  #@param {type: "string"}
gpu_enabled = True #@param {type:"boolean"}
cpu_enabled = False #@param {type:"boolean"}
optix_enabled = False #@param {type:"boolean"}
#@markdown ---


In [2]:
!chcp 65001
!dir

Active code page: 65001
 Volume in drive D is 雑用
 Volume Serial Number is D4AC-1350

 Directory of d:\OneDrive - 日本大学\NihonUniv\Jupyter\marume\graphic\第12回

2022/07/05  00:41    <DIR>          .
2022/07/04  15:08    <DIR>          ..
2022/07/04  16:44    <DIR>          .ipynb_checkpoints
2022/07/05  00:27               120 1.sh
2022/07/05  00:27               101 2.sh
2022/07/05  00:41    <DIR>          blender3.2.0
2022/06/08  21:13       192,843,500 blender3.2.0.tar.bz2
2022/07/04  15:07    <DIR>          Ex11_XToon
2022/07/04  16:07    <DIR>          Ex12_PostStylization
2022/07/04  15:16    <DIR>          images
2022/07/05  00:54            22,041 Pr12-2_blender_rendering.ipynb
2022/07/04  15:17         1,557,464 Pr12_1_LightPath.blend
2021/06/30  08:40         1,543,264 Pr12_1_LightPath.blend1
2022/07/05  00:47               633 setgpu.py
               7 File(s)    195,967,123 bytes
               7 Dir(s)  210,198,290,432 bytes free


## blenderのインストール

In [3]:
import os

os.environ["LD_PRELOAD"] = ""

!apt remove libtcmalloc-minimal4
!apt install libtcmalloc-minimal4

os.environ["LD_PRELOAD"] = "/usr/lib/x86_64-linux-gnu/libtcmalloc_minimal.so.4.3.0"

'apt' is not recognized as an internal or external command,
operable program or batch file.
'apt' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
gpu = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print("Current GPU: " + gpu[0])

if gpu[0] == "Tesla K80" and optix_enabled:
  print("OptiX disabled because of unsupported GPU")
  optix_enabled = False

Current GPU: NVIDIA GeForce RTX 3060 Ti


In [5]:
blender_version = 'blender3.2.0'
download_path="https://ftp.nluug.nl/pub/graphics/blender/release/Blender3.2/blender-3.2.0-linux-x64.tar.xz"

In [6]:
!rm -r $blender_version
!mkdir $blender_version
if blender_version == "blender2.81" or "blender2.80":
    !wget -O '{blender_version}.tar.bz2' -nc $download_path
    !tar -xf '{blender_version}.tar.bz2' -C ./$blender_version --strip-components=1
else:
    !wget -O '{blender_version}.tar.xz' -nc $download_path
    !tar xf '{blender_version}.tar.xz' -C ./$blender_version --strip-components=1


'rm' is not recognized as an internal or external command,
operable program or batch file.
A subdirectory or file blender3.2.0 already exists.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open ''blender3.2.0.tar.bz2''


In [7]:
# Enable GPU rendering (or add custom properties here)
data = "import re\n"+\
    "import bpy\n"+\
    "scene = bpy.context.scene\n"+\
    "scene.cycles.device = 'GPU'\n"+\
    "prefs = bpy.context.preferences\n"+\
    "prefs.addons['cycles'].preferences.get_devices()\n"+\
    "cprefs = prefs.addons['cycles'].preferences\n"+\
    "print(cprefs)\n"+\
    "for compute_device_type in ('CUDA', 'OPENCL', 'NONE'):\n"+\
    "    try:\n"+\
    "        cprefs.compute_device_type = compute_device_type\n"+\
    "        print('Device found:',compute_device_type)\n"+\
    "        break\n"+\
    "    except TypeError:\n"+\
    "        pass\n"+\
    "for device in cprefs.devices:\n"+\
    "    if not re.match('intel', device.name, re.I):\n"+\
    "        print('Activating',device)\n"+\
    "        device.use = "+str(gpu_enabled)+"\n"+\
    "    else:\n"+\
    "        device.use = "+str(cpu_enabled)+"\n"
with open('setgpu.py', 'w') as f:
    f.write(data)

renderer = "CUDA"
if optix_enabled:
    print("Note: You're currently using OptiX renderer. If an error occurred, the current GPU (example: Tesla K80) is not supported and you need to switch back to Cycles.")
    renderer = "OPTIX"

## Render Animation (アニメーションをレンダリング)

全フレームをレンダリング

```
!sudo ./$blender_version/blender -P './setgpu.py'  -b '{path_to_blend}' -E CYCLES -o '{output_path}' -a
```

1から120までを10とばしでレンダリング
- -s: フレーム開始番号
- -e: フレーム終了番号
- -j: レンダリングフレーム間隔
```
!sudo ./$blender_version/blender -P './setgpu.py'  -b '{path_to_blend}' -E CYCLES -o '{output_path}' -s 1 -e 120 -j 10 -a
```

In [8]:
!sudo ./$blender_version/blender -P './setgpu.py'  -b '{path_to_blend}' -E CYCLES -o '{output_path}' -s 1 -e 120 -j 30 -a

## Render Image (1フレームをレンダリング)

{frame}番号を指定してレンダリングできる．

```
!sudo ./$blender_version/blender -P 'setgpu.py' -b '{path_to_blend}' -E CYCLES -o '{output_path}' -f {frame}
```

50フレーム目を指定する場合，


```
!sudo ./$blender_version/blender -P 'setgpu.py' -b '{path_to_blend}' -E CYCLES -o '{output_path}' -f 50
```

のようにすればレンダリングできる．





In [9]:
!sudo ./$blender_version/blender -P 'setgpu.py' -b '{path_to_blend}' -E CYCLES -o '{output_path}' -f 2

## 出力画像の確認

In [10]:
from IPython.display import Image

Image(filename='images/normal0002.png')

FileNotFoundError: [Errno 2] No such file or directory: 'images/normal0002.png'

## Googleドライブの利用

以下のコードでGoogleドライブのデータを入出力できるようになります．

Googleドライブがマウントされると，
- `/gdrive/My Drive/{file_path}`

のようにしてデータにアクセスできます．


In [ ]:
from google.colab import drive
drive.mount('/gdrive')

: 

: 

: 